Импортируем необходимые пакеты

In [3]:
import numpy as np
import math
from functools import reduce
from copy import deepcopy

**Необходимые функции квантового симмулятора**

Базис системы кубитов

In [4]:
qubit01 = {0: np.array([1 , 0 ]),1: np.array([0 , 1])}

Тензор системы кубит

In [5]:
#перевод системы кубит в общий вектор
#ввод(list) - список кубит(0 или 1)
def tensorqubit(list):
  q = reduce((lambda x,y: np.kron(x,y)), list)
  return q

Адамар + XNOT + INOT(Можно еще комплексную дописать и еще NOT -ы ввести)

In [6]:
H = np.array([[1/math.sqrt(2),1/math.sqrt(2)],[1/math.sqrt(2),-1/math.sqrt(2)]])
X = np.array([[0,1],[1,0]])
I = np.array([[1,0],[0,1]])

i номер кубита в битовую запись размера n

In [7]:
# Перевод i кубита по счету в битовую запись размера n
# Ввод (n) - количество кубит, (i) - номер кубита 
# вывод int запись бита
def inttobit(n, i):
    return list(map(int, list(format(i, "0{0}b".format(n)))))

Функция CNOT для системы n кубит (контролирующий, контролируемый, n кубит){для тоффоли нужно еще цикл внутри писать} 

In [8]:
# Находит матрицу CNOT в системе n кубит
# Ввод: (q1) - номер контролирующего кубита, (q2) - номер контролируемого кубита, (n) - всего кубит в системе  
# Вывод - матрица 4*4 (пока без комплексной части)
def CNOT(q1, q2, n):
  matrixCNOT = np.zeros((2 ** n, 2 ** n))
  for i in range(2 ** n):
    binary = inttobit(n, i)
    matrixCNOT[:, i] =reduce(np.kron, [
        [qubit01[k]] if j != q2 else qubit01[((binary[q1 - 1])^ (binary[q2 - 1]))] \
        for j, k in enumerate(binary, start=1)])
  return matrixCNOT

**ДОЙЧА - ЙОЖИ**

**Новые необходимые функции**

функция i шага, но через лист(возникли проблемы перевода tuple поэтому пришлось поменять чуть функцию)

In [9]:
# Находит матрицу унитарных операторов i-того шага
# Ввод - Список унитарных операторов в этом шаге(List)
# Вывод - матрица такого шага n**2 * n**2
def UnitStep(list):
  Step_i = reduce((lambda x,y: np.kron(x,y)), list)
  return Step_i

функция цепи, но через лист(возникли проблемы перевода tuple поэтому пришлось поменять чуть функцию)

In [10]:
# Преобразование системы кубит после гейтовой цепочки
# Ввод (Stepsnumber) - количество шагов в цепи, (qubitsstart) - начальное состояние кубит,(list) - список матриц общего преобразования на i-том шаге
# Вывод - Состояние кубитов после прохождение цепи (вектор n**2)
def Chainqubit(qubitsstart, list):
  for i in range(len(list)):
    qubitsstart = list[i].dot(qubitsstart)
  return qubitsstart

Функции алгоритма Дойча - Йожа (сбалансированная и константа)

In [11]:
def balanceOracul(n):

  randombalance = np.random.randint(1,2**n)
  randombalancebit = inttobit(n, randombalance)
  XNOTCHAIN = []
  CNOTCHAIN = []
  CHAINORACULE = []
  for i in range(n):
    if randombalancebit[i] == 1:
      XNOTCHAIN.append(X)
    else:
      XNOTCHAIN.append(I)
  XNOTCHAIN.append(I)
  CHAINORACULE.append(UnitStep(XNOTCHAIN))
  for j in range(n):
    CHAINORACULE.append(CNOT(n - j,n+1,n+1))
  
  CHAINORACULE.append(UnitStep(XNOTCHAIN))

  return CHAINORACULE

def constant(n):
  randomconst = np.random.randint(2)
  const=[]
  for i in range(n):
    const.append(I)
  if randomconst == 1:
    const.append(X)
  else:
    const.append(I)
  return UnitStep(const)
    


n гейтов адамара

In [12]:
def Hn(n):
  H_n = []
  for i  in range(n):
    H_n.append(H)
  return UnitStep(H_n)

Задаем количество кубит

In [13]:
n = int(input())
m1 = np.array([1, 0])
m2 = np.array([0, 1])
systemQubit = []
for i in range(n):
  systemQubit.append(m1)
systemQubit.append(m2)
systemQubit

3


[array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1])]

общий вектор кубит

In [14]:
qubit = tensorqubit(systemQubit)

Задает гейтовую цепочку. Для сбалансированной - balance, для константной - const

In [15]:
FullChain = []
FullChain.append(Hn(n+1))
indicator = input()
if indicator == 'balance':
  Oracul = balanceOracul(n)
  for i in range(2+n):
    FullChain.append(Oracul[i])
if indicator == 'const':
  Oracul = constant(n)
  FullChain.append(Oracul)
FullChain.append(np.kron(Hn(n),I))

balance


Выводит результат(сбалансированная или константа){потом нужно как то поинтереснее реализовать черный ящик}

In [18]:
o = Chainqubit(qubit, FullChain)
o = o*o
indexnonzero = np.where(o > 0.001)
print((bin(indexnonzero[0][0])[2:].zfill(n+1))[:n])
print((bin(indexnonzero[0][1])[2:].zfill(n+1))[:n])

111
111
[1, 1, 1, 0]
[1, 1, 1, 1]


Если первые n кубита = 0 => константная

Если первые n кубита = 1 => Сбалансированная

**